<a href="https://colab.research.google.com/github/Al1952/Final_Exam_Data_Analysis_with_Python/blob/master/Using_the_TFLite_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  # tensorflow_version only exists in Colab
   %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
ten = Dense(units =1, input_shape = [1])
model = Sequential(Dense(units =1, input_shape = [1]))
model.compile (optimizer = 'sgd', loss = 'mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0,2.0,3.0,4.0],dtype=float)
ys = np.array([-3.0,-1.0,1.0,3.0,5.0,7.0], dtype =  float)

model.fit(xs,ys, epochs=500)


In [ ]:
print(model.predict([10.0]))
#print ('here is what I learned: {}'.format(ten.get_weights()))

1/1 [==============================] - 0s 44ms/step
[[18.977083]]
here is what I learned: []


In [ ]:
# Save the model and put it into the directory

export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

In [ ]:
#convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)  # takes data from the saved model in directory
tflite_model =  converter.convert()  # converts it into tflite


In [ ]:
import pathlib
tflite_model_file =  pathlib.Path('model.tflite')
tflite_model_file.write_bytes (tflite_model)

# gives the size of the file

1092

In [ ]:
# Load TFLite model and allocate tensors.
# uses python stand alone interpreter for tflite
interpreter = tf.lite.Interpreter (model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details =  interpreter.get_output_details()
print (input_details )
print (output_details )


[{'name': 'serving_default_dense_3_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
to_predict = np.array ([[10.0]],dtype = np.float32)
print ('to_predict',to_predict)

# this line sets the value of the tensor to predict to 10
interpreter.set_tensor (input_details[0] ['index'],to_predict)

# invoke the interpreter
interpreter.invoke()

# read the results
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print ('tflite_results', tflite_results)

# so we passed in the value '10', and got out the value '18.977083'


#  So it shows that tensorflow lite is giving the same output as tensor flow

to_predict [[10.]]
tflite_results [[18.977083]]
